In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
%config InlineBackend.figure_format = 'retina'

In [2]:
data_path = 'C:/Users/CRH/Desktop/cityu/huawei/Time Series Forecasting/Small Sample Time Series Forecasting/Time_Series'
fnames = [fname for fname in os.listdir(data_path) if os.path.isfile(os.path.join(data_path, fname))]
print(len(fnames), "files")

1146 files


In [3]:
def process_diagonal_with_last_1_m0():
    data_full = []
    for fname in fnames:
        data = pd.read_csv(os.path.join(data_path, fname), index_col=0)
        data = data.drop([' item'], axis=1)
        data.columns = ['qty', 'qty1', 'qty2', 'qty3']
        data['x1_for_m0'] = data['qty1']
        data['x2_for_m0'] = data['qty2'].shift(1, axis=0)
        data['x3_for_m0'] = data['qty3'].shift(2, axis=0)
        data['x1_for_m1'] = data['qty2']
        data['x2_for_m1'] = data['qty3'].shift(2, axis=0)
        data['x1_for_m2'] = data['qty3']
        data['Last_M0'] = data['qty'].shift(1, axis=0)
        data['M0'] = data['qty']
        data['M1'] = data['qty'].shift(-1, axis=0)
        data['M2'] = data['qty'].shift(-2, axis=0)
        data['M3'] = data['qty'].shift(-3, axis=0)
        data = data.drop(['qty', 'qty1', 'qty2', 'qty3'], axis=1)
        data = data.dropna(how='any', axis=0)

        temp = np.zeros((42, 11))  # this should be modified
        for i in range(1215, 1257):
            try:
                temp[i-1215] = data.loc[i].values
            except:
                continue
        data_full.append(temp)
    return data_full

In [4]:
# data_full = process_original_with_last_1_m0()
data_full = process_diagonal_with_last_1_m0()

In [5]:
def split_m0(past_days):
#     split_ind = 38  # if original
    split_ind = 36  # if digonal
    X_train, Y_train = [], []
    X_test, Y_test = [], []

    for ind in range(past_days, split_ind):
        for item_ind in range(len(fnames)):
            X_train.append(data_full[item_ind][ind - past_days: ind + 1][:,:-4])
            Y_train.append(data_full[item_ind][ind][-4]) # change the number of y

    for ind in range(split_ind, split_ind + 6):
        for item_ind in range(len(fnames)):
            X_test.append(data_full[item_ind][ind - past_days: ind + 1][:,:-4])
            Y_test.append(data_full[item_ind][ind][-4]) # change the number of y
    X_train, Y_train = np.array(X_train), np.array(Y_train)
    X_test, Y_test = np.array(X_test), np.array(Y_test)
    if past_days == 0:
        X_train, Y_train, X_test, Y_test = \
        np.squeeze(X_train), np.squeeze(Y_train), np.squeeze(X_test), np.squeeze(Y_test) 
    print("The shape of X_train:", X_train.shape)
    print("The shape of Y_train:", Y_train.shape)
    print("The shape of X_test:", X_test.shape)
    print("The shape of Y_train:", Y_test.shape)
    return X_train, Y_train, X_test, Y_test

X_train, Y_train, X_test, Y_test = split_m0(0) 
Y_test=Y_test.reshape(-1,1)

The shape of X_train: (41256, 7)
The shape of Y_train: (41256,)
The shape of X_test: (6876, 7)
The shape of Y_train: (6876,)


In [8]:
def acc_out(item_index):
    item_ytest = []
    item_ypred = []
    count = 0
    for i in range(6):
        item_ytest.append(Y_test[item_index+count*1146])
        item_ypred.append(Y_pred_test[item_index+count*1146])
        count += 1
    acc = []
    for i in range(6):
        for index, value in enumerate(item_ypred[i]): 
            if  max(item_ytest[i][index], item_ypred[i][index]) == 0:
                acc.append(np.nan)
                continue
            acc.append(np.divide(min(item_ytest[i][index], item_ypred[i][index]),
                                 max(item_ytest[i][index], item_ypred[i][index])))
    acc = np.array(acc)
    acc = acc.reshape(6, 1)
    return acc

# Prediction of M0

In [19]:
import statsmodels.api as sm

X_train=sm.add_constant(X_train)
X_test=sm.add_constant(X_test)
model=sm.OLS(Y_train,X_train).fit()
Y_pred_test = model.predict(X_test)
Y_pred_test=Y_pred_test.reshape(-1,1)

acc_list = []
for i in range(1146):
    acc_list.append(acc_out(i))
acc_list = np.array(acc_list)
print(np.nanmean(acc_list,axis=0))

print_model = model.summary()
print(print_model)

[[0.62084974]
 [0.70827295]
 [0.69174722]
 [0.67712707]
 [0.6974861 ]
 [0.67774241]]
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.821
Model:                            OLS   Adj. R-squared:                  0.821
Method:                 Least Squares   F-statistic:                 3.158e+04
Date:                Tue, 23 Jun 2020   Prob (F-statistic):               0.00
Time:                        23:08:40   Log-Likelihood:                 34317.
No. Observations:               41256   AIC:                        -6.862e+04
Df Residuals:                   41249   BIC:                        -6.856e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------

# Prediction of M1

In [22]:
def split_m1(past_days):
#     split_ind = 38  # if original
    split_ind = 36  # if digonal
    X_train, Y_train = [], []
    X_test, Y_test = [], []

    for ind in range(past_days, split_ind):
        for item_ind in range(len(fnames)):
            X_train.append(data_full[item_ind][ind - past_days: ind + 1][:,:-4])
            Y_train.append(data_full[item_ind][ind][-3]) # change the number of y

    for ind in range(split_ind, split_ind + 6):
        for item_ind in range(len(fnames)):
            X_test.append(data_full[item_ind][ind - past_days: ind + 1][:,:-4])
            Y_test.append(data_full[item_ind][ind][-3]) # change the number of y
    X_train, Y_train = np.array(X_train), np.array(Y_train)
    X_test, Y_test = np.array(X_test), np.array(Y_test)
    if past_days == 0:
        X_train, Y_train, X_test, Y_test = \
        np.squeeze(X_train), np.squeeze(Y_train), np.squeeze(X_test), np.squeeze(Y_test) 
    print("The shape of X_train:", X_train.shape)
    print("The shape of Y_train:", Y_train.shape)
    print("The shape of X_test:", X_test.shape)
    print("The shape of Y_train:", Y_test.shape)
    return X_train, Y_train, X_test, Y_test

X_train, Y_train, X_test, Y_test = split_m1(0) 
Y_test=Y_test.reshape(-1,1)

The shape of X_train: (41256, 7)
The shape of Y_train: (41256,)
The shape of X_test: (6876, 7)
The shape of Y_train: (6876,)


In [23]:
X_train=sm.add_constant(X_train)
X_test=sm.add_constant(X_test)
model=sm.OLS(Y_train,X_train).fit()
Y_pred_test = model.predict(X_test)
Y_pred_test=Y_pred_test.reshape(-1,1)

acc_list = []
for i in range(1146):
    acc_list.append(acc_out(i))
acc_list = np.array(acc_list)
print(np.nanmean(acc_list,axis=0))

print_model = model.summary()
print(print_model)

[[0.58496147]
 [0.59918812]
 [0.49490029]
 [0.59278778]
 [0.58678644]
 [0.56544121]]
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.420
Model:                            OLS   Adj. R-squared:                  0.420
Method:                 Least Squares   F-statistic:                     4978.
Date:                Tue, 23 Jun 2020   Prob (F-statistic):               0.00
Time:                        23:10:20   Log-Likelihood:                 10009.
No. Observations:               41256   AIC:                        -2.000e+04
Df Residuals:                   41249   BIC:                        -1.994e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------

# Prediction of M2

In [24]:
def split_m2(past_days):
#     split_ind = 38  # if original
    split_ind = 36  # if digonal
    X_train, Y_train = [], []
    X_test, Y_test = [], []

    for ind in range(past_days, split_ind):
        for item_ind in range(len(fnames)):
            X_train.append(data_full[item_ind][ind - past_days: ind + 1][:,:-4])
            Y_train.append(data_full[item_ind][ind][-2]) # change the number of y

    for ind in range(split_ind, split_ind + 6):
        for item_ind in range(len(fnames)):
            X_test.append(data_full[item_ind][ind - past_days: ind + 1][:,:-4])
            Y_test.append(data_full[item_ind][ind][-2]) # change the number of y
    X_train, Y_train = np.array(X_train), np.array(Y_train)
    X_test, Y_test = np.array(X_test), np.array(Y_test)
    if past_days == 0:
        X_train, Y_train, X_test, Y_test = \
        np.squeeze(X_train), np.squeeze(Y_train), np.squeeze(X_test), np.squeeze(Y_test) 
    print("The shape of X_train:", X_train.shape)
    print("The shape of Y_train:", Y_train.shape)
    print("The shape of X_test:", X_test.shape)
    print("The shape of Y_train:", Y_test.shape)
    return X_train, Y_train, X_test, Y_test

X_train, Y_train, X_test, Y_test = split_m2(0) 
Y_test=Y_test.reshape(-1,1)

The shape of X_train: (41256, 7)
The shape of Y_train: (41256,)
The shape of X_test: (6876, 7)
The shape of Y_train: (6876,)


In [25]:
X_train=sm.add_constant(X_train)
X_test=sm.add_constant(X_test)
model=sm.OLS(Y_train,X_train).fit()
Y_pred_test = model.predict(X_test)
Y_pred_test=Y_pred_test.reshape(-1,1)

acc_list = []
for i in range(1146):
    acc_list.append(acc_out(i))
acc_list = np.array(acc_list)
print(np.nanmean(acc_list,axis=0))

print_model = model.summary()
print(print_model)

[[0.55641928]
 [0.48493321]
 [0.56641321]
 [0.56795342]
 [0.56969181]
 [0.55267561]]
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.338
Model:                            OLS   Adj. R-squared:                  0.338
Method:                 Least Squares   F-statistic:                     3508.
Date:                Tue, 23 Jun 2020   Prob (F-statistic):               0.00
Time:                        23:11:24   Log-Likelihood:                 7208.3
No. Observations:               41256   AIC:                        -1.440e+04
Df Residuals:                   41249   BIC:                        -1.434e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------

# Prediction of M3

In [26]:
def split_m3(past_days):
#     split_ind = 38  # if original
    split_ind = 36  # if digonal
    X_train, Y_train = [], []
    X_test, Y_test = [], []

    for ind in range(past_days, split_ind):
        for item_ind in range(len(fnames)):
            X_train.append(data_full[item_ind][ind - past_days: ind + 1][:,:-4])
            Y_train.append(data_full[item_ind][ind][-1]) # change the number of y

    for ind in range(split_ind, split_ind + 6):
        for item_ind in range(len(fnames)):
            X_test.append(data_full[item_ind][ind - past_days: ind + 1][:,:-4])
            Y_test.append(data_full[item_ind][ind][-1]) # change the number of y
    X_train, Y_train = np.array(X_train), np.array(Y_train)
    X_test, Y_test = np.array(X_test), np.array(Y_test)
    if past_days == 0:
        X_train, Y_train, X_test, Y_test = \
        np.squeeze(X_train), np.squeeze(Y_train), np.squeeze(X_test), np.squeeze(Y_test) 
    print("The shape of X_train:", X_train.shape)
    print("The shape of Y_train:", Y_train.shape)
    print("The shape of X_test:", X_test.shape)
    print("The shape of Y_train:", Y_test.shape)
    return X_train, Y_train, X_test, Y_test

X_train, Y_train, X_test, Y_test = split_m3(0) 
Y_test=Y_test.reshape(-1,1)

The shape of X_train: (41256, 7)
The shape of Y_train: (41256,)
The shape of X_test: (6876, 7)
The shape of Y_train: (6876,)


In [27]:
X_train=sm.add_constant(X_train)
X_test=sm.add_constant(X_test)
model=sm.OLS(Y_train,X_train).fit()
Y_pred_test = model.predict(X_test)
Y_pred_test=Y_pred_test.reshape(-1,1)

acc_list = []
for i in range(1146):
    acc_list.append(acc_out(i))
acc_list = np.array(acc_list)
print(np.nanmean(acc_list,axis=0))

print_model = model.summary()
print(print_model)

[[0.44996456]
 [0.55185305]
 [0.55075342]
 [0.55005702]
 [0.5636868 ]
 [0.50481337]]
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.263
Model:                            OLS   Adj. R-squared:                  0.263
Method:                 Least Squares   F-statistic:                     2449.
Date:                Tue, 23 Jun 2020   Prob (F-statistic):               0.00
Time:                        23:12:34   Log-Likelihood:                 4933.7
No. Observations:               41256   AIC:                            -9853.
Df Residuals:                   41249   BIC:                            -9793.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------